# Задание
Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты. 

## Этапы выполнения

1. Выделить три столбца-признака для обучения, выбранные RFE.
2. Выделить три столбца-признака для обучения, выбранные SelectKBest.
3. Обучить регрессию на первых трёх столбцах, оценить качество модели на тесте.
4. Обучить регрессию на вторых трёх столбцах, оценено качество модели на тесте.
5. Произвести сравнение выбранных метрик в форме комментария. Дать ответ на вопрос «Какой метод отбора признаков показал наилучший результат на тестовой выборке?»

In [64]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [65]:
#импортируем данные
data = pd.read_excel('data/data_ford_price.xlsx') 

In [66]:
#используем предобработку данных, удаляем пропуски, отделяем результирующий признак от остальных
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)
y = data['price']
x = data.drop(columns='price')

In [67]:
#делим датасет на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

### RFE метод

In [68]:
from sklearn.feature_selection import RFE

In [69]:
#выбираем три признака для модели
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()
#'year', 'cylinders', 'lat'

array(['year', 'cylinders', 'lat'], dtype=object)

In [70]:
#оставляем только нужные столбцы
x = data[['year', 'cylinders', 'lat']]

#делим датасет на тренировочную и тестовую части с учётом нового x
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

#оцениваем метрики
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)
#MAE: 5206.677

MAE: 5206.677


### SelectKBest

Т.к. стоит задача регрессии, то выходящие признаки будут числовыми, поэтому в качестве скорринговой функции используется функция корреляции. В данном случае это корреляция Пирсона, т.к. большинстов признаков у нас - числовые.

In [71]:
#импортируем SelectKBest, корреляцию Пирсона
from sklearn.feature_selection import SelectKBest, f_regression

In [72]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)
y = data['price']
x = data.drop(columns='price')

#делим датасет на тренировочную и тестовую части
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# выбираем три признака
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()
#'year', 'cylinders', 'odometer'

array(['year', 'cylinders', 'odometer'], dtype=object)

In [73]:
#оставляем только нужные столбцы
x = data[['year', 'cylinders', 'odometer']]

#делим датасет на тренировочную и тестовую части с учётом нового x
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)

#оцениваем метрику
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)
#MAE: 4800.916

MAE: 4800.916


## Вывод
По результатам выбора признаков различными методами лучший результат на тестовой выборке показал метод SelectKBest. В нем значение метрики MAE ниже (4800 против 5206 у метода RFE). При этом отличие в результатах методов было только в одном признаке: SelectKBest поставил признак 'odometer' на третье место, а RFE - поставил 'lat'